In [ ]:
# Importing all the necessary libraries
import sys
import os
from PIL import Image
import time

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
import customtkinter

from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)  # type: ignore
from matplotlib.figure import Figure
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
from matplotlib.widgets import SpanSelector
from matplotlib.backend_bases import key_press_handler

import numpy as np

from BaselineRemoval import BaselineRemoval

import random

import serial

import cv2

In [ ]:
# Modes: "System" (standard), "Dark", "Light"
customtkinter.set_appearance_mode("Dark")
# Themes: "blue" (standard), "green", "dark-blue"
customtkinter.set_default_color_theme("blue")
# customtkinter.deactivate_automatic_dpi_awareness()

class SerialConsole(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)
        
class CameraView(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)
        
class ScanSettings(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)
        
class CNCButtons(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)

class App(customtkinter.CTk):
    def __init__(self):
        super().__init__()
        self.protocol("WM_DELETE_WINDOW", self.OnQuitApp)
        self.title("LST Raman Microscope Scanner")
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        app_window_width = 1280
        app_window_height = 720
        self.geometry(
            f"{app_window_width}x{app_window_height}+{int(screen_width/2-app_window_width/2)}+{int(screen_height/2-app_window_height/2)}")
        self.minsize(app_window_width, app_window_height)
        self.initialdir = os.getcwd()
        self.icons_folder = 'icons'

        # current_folder = os.getcwd()
        current_folder = globals()['_dh'][0]
        image_path = os.path.join(self.initialdir, self.icons_folder)
        try:
            self.iconbitmap(os.path.join(image_path, "lst_logo.ico"))
        except:
            pass


    def OnQuitApp(self):
        quit_app_window = customtkinter.CTkToplevel(self)
        quit_app_window.geometry(
            f"320x100+{self.winfo_x()+self.winfo_width()//2-160}+{self.winfo_y()+self.winfo_height()//2-50}")
        quit_app_window.title("Quit Application")
        quit_app_window.resizable(False, False)
        quit_app_window.deiconify()

        label = customtkinter.CTkLabel(
            quit_app_window, text="Are you sure you want to quit?")
        label.grid(row=0, column=0, columnspan=2, padx=10, pady=10)
        yes_button = customtkinter.CTkButton(
            quit_app_window, text="Yes", command=self._quit)
        yes_button.grid(row=1, column=0, padx=10, pady=10)
        no_button = customtkinter.CTkButton(
            quit_app_window, text="No", command=quit_app_window.destroy)
        no_button.grid(row=1, column=1, padx=10, pady=10)

    def _quit(self):
        self.quit()
        self.destroy()


if __name__ == "__main__":
    app = App()
    app.mainloop()